# Goal of script

Created on 01/03/2023 by Claire S

This script assesses the cleaned data in terms of split emails and new features created in the following scripts:

- Step0_Data_Extraction
- Step1_Regex_Data_Cleaning

This Script filters the data and tries to find more targets

In [108]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
import pandas as pd
import pickle
import numpy as np

In [110]:
# pandas options
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [111]:
# try and import my own functions:
import sys
import ipynb
import re
sys.path.append('/project/Xelix_Project/utils')

In [113]:
# go through this for each run
this_file = "data/Step1_Data/labelled_emails_Step1_Split_newfeatures_V2.pkl"
# this_file = "data/labelled_emails_more_features_checked.csv"

In [114]:
# go through this for each run
# df = pd.read_csv(this_file)
df = pd.read_pickle(this_file)

In [115]:
# df.to_csv('data/labelled_emails_V2.csv')

In [116]:
# df = pd.read_pickle("data/Step1_Data/unlabelled_emails_Step1_Split_newfeatures_run2.pkl")

print(len(df))

17963


In [117]:
print(len(df.columns))
df.columns

34


Index(['iter_count_list', 'email_id', 'is_part_of_thread', 'num_in_thread',
       'thread_id', 'initial_domain', 'from_address', 'from_name', 'sent_to',
       'sent_to_how_many', 'bcc', 'cc', 'reply_to_address', 'internal_date',
       'category_id', 'category_slug', 'has_sort_code', 'sort_code',
       'has_attachment', 'mentions_bank', 'mentions_change',
       'mentions_account_specifics', 'mentions_bank_account',
       'subject_mentions_bank', 'subject_mentions_change',
       'attachment_mentions_bank', 'mentions_statement', 'mentions_invoice',
       'mentions_fraud', 'has_attention_ext_flag', 'has_alert_attach_flag',
       'subject', 'attachments', 'email_text_body'],
      dtype='object')

## Some manual recoding of true bank change emails


In [118]:
bc_ids = [
186977,
186977,
282436,
283345,
284818,
287342,
291518,
291520,
292234,
292234,
292234,
349649,
351685,
356414,
380734,
380829
]

thread_ids = [
2,
1,
0,
0,
0,
0,
0,
0,
4,
3,
1,
0,
0,
0,
7,
2
]

matches = pd.DataFrame({'email_id':bc_ids,'num_in_thread':thread_ids})
matches['final_label'] = 1
matches

,email_id,num_in_thread,final_label
0,186977,2,1
1,186977,1,1
2,282436,0,1
3,283345,0,1
4,284818,0,1
5,287342,0,1
6,291518,0,1
7,291520,0,1
8,292234,4,1
9,292234,3,1


In [119]:
# Merge the two dataframes on the common columns
# df['final_label'] = 0
merged_df = df.merge(matches, on=['email_id', 'num_in_thread'], how='left')
merged_df['final_label'] = merged_df['final_label'].fillna(0).astype(int)

merged_df.columns

Index(['iter_count_list', 'email_id', 'is_part_of_thread', 'num_in_thread',
       'thread_id', 'initial_domain', 'from_address', 'from_name', 'sent_to',
       'sent_to_how_many', 'bcc', 'cc', 'reply_to_address', 'internal_date',
       'category_id', 'category_slug', 'has_sort_code', 'sort_code',
       'has_attachment', 'mentions_bank', 'mentions_change',
       'mentions_account_specifics', 'mentions_bank_account',
       'subject_mentions_bank', 'subject_mentions_change',
       'attachment_mentions_bank', 'mentions_statement', 'mentions_invoice',
       'mentions_fraud', 'has_attention_ext_flag', 'has_alert_attach_flag',
       'subject', 'attachments', 'email_text_body', 'final_label'],
      dtype='object')

In [120]:
sum(merged_df['final_label'])

18

In [121]:
df = merged_df
sum(df['final_label'])

18

In [122]:
df.head(10)

,iter_count_list,email_id,is_part_of_thread,num_in_thread,thread_id,initial_domain,from_address,from_name,sent_to,sent_to_how_many,bcc,cc,reply_to_address,internal_date,category_id,category_slug,has_sort_code,sort_code,has_attachment,mentions_bank,mentions_change,mentions_account_specifics,mentions_bank_account,subject_mentions_bank,subject_mentions_change,attachment_mentions_bank,mentions_statement,mentions_invoice,mentions_fraud,has_attention_ext_flag,has_alert_attach_flag,subject,attachments,email_text_body,final_label
0,1,167529,False,0,False,bluestemgroup.co.uk,sales@bluestemgroup.co.uk,sales@bluestemgroup.co.uk,{Purchase.Ledger@evo-group.co.uk},1,{},{},{},2021-11-17 16:08:00+00,100,invoices,False,None,False,False,False,False,False,False,False,False,False,False,False,True,False,BLUESTEM GROUP INV PP2422423 - Order X843348. ...,None,Attention: This email originated outside of ...,0
1,2,167544,False,0,False,albion.co.uk,sefika@albion.co.uk,sefika@albion.co.uk,{Matt.Hales@evo-group.co.uk},1,{},{Ryan.Leigh@evo-group.co.uk},{},2022-07-05 15:37:48+00,1,statement,False,None,True,False,False,False,False,False,False,False,True,False,False,True,False,"Overdue Account £59,632.58",Followups(6).pdf,Attention: This email originated outside of ...,0
2,3,170741,True,10,170741_t,evo-group.co.uk,Lisa.Dyson@evo-group.co.uk,Lisa.Dyson@evo-group.co.uk,{Purchase.Ledger@evo-group.co.uk},1,None,None,{},2021-07-28 10:10:32+00,100,invoices,False,None,False,False,False,False,False,False,False,False,False,False,False,False,False,FW: APRIL ACCOUNT,None,Lisa Dyson Group Accounts Payable Supervi...,0
3,3,170741,True,9,170741_t,evo-group.co.uk,Peter Bown <peter.bown@slingsby.com>,Peter Bown <peter.bown@slingsby.com>,[Lisa.Dyson@evo-group.co.uk],1,None,None,{},28 July 2021 10:53,100,invoices,False,None,False,False,False,False,False,False,False,None,False,True,False,True,False,RE: APRIL ACCOUNT,None,Attention: This email originated outside of...,0
4,3,170741,True,8,170741_t,evo-group.co.uk,Lisa Dyson < Lisa.Dyson@evo-group.co.uk >,Lisa Dyson < Lisa.Dyson@evo-group.co.uk >,[ peter.bown@slingsby.com ],1,None,None,{},28 July 2021 10:35,100,invoices,False,None,False,False,False,False,False,False,False,None,False,False,False,False,False,RE: APRIL ACCOUNT,None,Thanks Peter Could you send me the credit...,0
5,3,170741,True,7,170741_t,evo-group.co.uk,Peter Bown < peter.bown@slingsby.com >,Peter Bown < peter.bown@slingsby.com >,[ Lisa.Dyson@evo-group.co.uk ],1,None,None,{},28 July 2021 10:31,100,invoices,False,None,False,False,False,False,False,False,False,None,True,False,False,True,False,RE: APRIL ACCOUNT,None,Attention: This email originated outside of...,0
6,3,170741,True,6,170741_t,evo-group.co.uk,Lisa Dyson < Lisa.Dyson@evo-group.co.uk >,Lisa Dyson < Lisa.Dyson@evo-group.co.uk >,[ peter.bown@slingsby.com ],1,None,None,{},28 July 2021 09:49,100,invoices,False,None,False,False,False,False,False,False,False,None,False,False,False,False,False,RE: APRIL ACCOUNT,None,"HI Pete Is our account on hold, if so is i...",0
7,3,170741,True,5,170741_t,evo-group.co.uk,Peter Bown < peter.bown@slingsby.com >,Peter Bown < peter.bown@slingsby.com >,"[ Lisa.Dyson@evo-group.co.uk , Matt.Hales@evo...",4,None,None,{},15 July 2021 14:43,100,invoices,False,None,False,False,True,False,False,False,False,None,True,False,False,True,False,FW: APRIL ACCOUNT,None,Attention: This email originated outside of...,0
8,3,170741,True,4,170741_t,evo-group.co.uk,Matt Hales < Matt.Hales@evo-group.co.uk >,Matt Hales < Matt.Hales@evo-group.co.uk >,"[ joanne.mcewan@slingsby.com , Lisa.Dyson@evo...",4,None,None,{},15 July 2021 14:19,100,invoices,False,None,False,False,False,False,False,False,False,None,False,False,False,False,False,RE: APRIL ACCOUNT,None,Hi Joanne Will look into as soon as I can;...,0
9,3,170741,True,3,170741_t,evo-group.co.uk,Joanne McEwan < joanne.mcewan@slingsby.com >,Joanne McEwan < joanne.mcewan@slingsby.com >,"[ Matt.Hales@evo-group.co.uk , morgan.morris@...",3,None,None,{},13 July 

In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17963 entries, 0 to 17962
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   iter_count_list             17963 non-null  int64 
 1   email_id                    17963 non-null  int64 
 2   is_part_of_thread           17963 non-null  bool  
 3   num_in_thread               17963 non-null  int64 
 4   thread_id                   17963 non-null  object
 5   initial_domain              17963 non-null  object
 6   from_address                17956 non-null  object
 7   from_name                   17956 non-null  object
 8   sent_to                     17956 non-null  object
 9   sent_to_how_many            17963 non-null  int64 
 10  bcc                         12901 non-null  object
 11  cc                          12901 non-null  object
 12  reply_to_address            17963 non-null  object
 13  internal_date               17956 non-null  ob

# Filter dataframe on conditions

In [125]:
df.head(1)

,iter_count_list,email_id,is_part_of_thread,num_in_thread,thread_id,initial_domain,from_address,from_name,sent_to,sent_to_how_many,bcc,cc,reply_to_address,internal_date,category_id,category_slug,has_sort_code,sort_code,has_attachment,mentions_bank,mentions_change,mentions_account_specifics,mentions_bank_account,subject_mentions_bank,subject_mentions_change,attachment_mentions_bank,mentions_statement,mentions_invoice,mentions_fraud,has_attention_ext_flag,has_alert_attach_flag,subject,attachments,email_text_body,final_label
0,1,167529,False,0,False,bluestemgroup.co.uk,sales@bluestemgroup.co.uk,sales@bluestemgroup.co.uk,{Purchase.Ledger@evo-group.co.uk},1,{},{},{},2021-11-17 16:08:00+00,100,invoices,False,None,False,False,False,False,False,False,False,False,False,False,False,True,False,BLUESTEM GROUP INV PP2422423 - Order X843348. ...,None,Attention: This email originated outside of ...,0


In [165]:
all_bchange = df.query('(final_label == 1)')
print(len(df.query('(final_label == 1)')))


18


In [171]:
# query = '''
# ((mentions_bank == True) | (mentions_bank_account == True) or (subject_mentions_bank == True)) & ((mentions_change == True) or (subject_mentions_change == True))
# '''
# new_df = df.query(query)
# print(len(new_df))

# query = '((mentions_bank) or (mentions_bank_account) or (subject_mentions_bank)) & ((mentions_change) or (subject_mentions_change))'
# new_df = df.query(query)
# print(len(new_df))
query = '((mentions_bank == True) or (mentions_bank_account == True) or (subject_mentions_bank == True)) & ((mentions_change == True) or (subject_mentions_change == True))'
new_df = df.query(query)
print(len(new_df))


347


In [167]:
# new_df = df.query('(mentions_bank == True) & (mentions_change == True)')
# print(len(new_df))

In [168]:
some_bchange = new_df.query('(final_label == 1)')
print(len(new_df.query('(final_label == 1)')))

16


In [163]:
i = 1
for row in all_bchange.itertuples():
    print("#" * 70, "\n\n")
    print(f'email index: {i}')
    i += 1
    print(f'subject line:\t{row.subject}')
    print(f'received from:\t{row.from_address}')
    print(f'is part of thread:\t{bool(row.is_part_of_thread)}')
    #print(f'num in thread:\t\t{int(row.num_in_thread)}')
    print(f'has attachment:\t\t{row.has_attachment}')
    print(f'has sort code:\t\t{row.has_sort_code}')
    print(f'mentions bank:\t\t{row.mentions_bank}')
    print(f'mentions b details:\t{row.mentions_account_specifics}')
    print(f'mentions change:\t{row.mentions_change}')
    print(f'subject mentions bank:\t{row.subject_mentions_bank}')
    print(f'subject mentions change:{row.subject_mentions_change}')
    #print(f'sent to:\t{row.sent_to}')
    print(f'sent to {row.sent_to_how_many} people')
    print(f'date sent:\t{row.internal_date}')
    print(f'slug:\t\t{row.category_slug}\n')
    print(row.email_text_body)

###################################################################### 


email index: 1
subject line:	FW: Tyronex Bank Account Change
received from:	Matt Tibble <Matt@tyronex.co.uk>  
is part of thread:	True
has attachment:		False
has sort code:		False
mentions bank:		True
mentions b details:	False
mentions change:	False
subject mentions bank:	True
subject mentions change:True
sent to 6 people
date sent:	13 September 2021 10:17  
slug:		bank-detail-change

   Alert: There is currently a serious threat from opening email attachments. If you have any concerns about an attached file please contact IT. Attention: This email originated outside of our organisation. Please be extra vigilant when opening attachments or clicking links.  Hi All  Apologies, please find attached the correct bank account details (sterling account).  Please confirm that these have been amended on your supplier account details.  Kind regards  Matt   Matthew Tibble | Senior Account Manager | E: matt@tyronex.co.uk DDI

In [164]:
i = 1
for row in some_bchange.itertuples():
    print("#" * 70, "\n\n")
    print(f'email index: {i}')
    i += 1
    print(f'subject line:\t{row.subject}')
    print(f'received from:\t{row.from_address}')
    print(f'is part of thread:\t{bool(row.is_part_of_thread)}')
    #print(f'num in thread:\t\t{int(row.num_in_thread)}')
    print(f'has attachment:\t\t{row.has_attachment}')
    print(f'has sort code:\t\t{row.has_sort_code}')
    print(f'mentions bank:\t\t{row.mentions_bank}')
    print(f'mentions b details:\t{row.mentions_account_specifics}')
    print(f'mentions change:\t{row.mentions_change}')
    print(f'subject mentions bank:\t{row.subject_mentions_bank}')
    print(f'subject mentions change:{row.subject_mentions_change}')
    #print(f'sent to:\t{row.sent_to}')
    print(f'sent to {row.sent_to_how_many} people')
    print(f'date sent:\t{row.internal_date}')
    print(f'slug:\t\t{row.category_slug}\n')
    print(row.email_text_body)

###################################################################### 


email index: 1
subject line:	FW: Tyronex Bank Account Change
received from:	Matt Tibble <Matt@tyronex.co.uk>  
is part of thread:	True
has attachment:		False
has sort code:		False
mentions bank:		True
mentions b details:	False
mentions change:	False
subject mentions bank:	True
subject mentions change:True
sent to 6 people
date sent:	13 September 2021 10:17  
slug:		bank-detail-change

   Alert: There is currently a serious threat from opening email attachments. If you have any concerns about an attached file please contact IT. Attention: This email originated outside of our organisation. Please be extra vigilant when opening attachments or clicking links.  Hi All  Apologies, please find attached the correct bank account details (sterling account).  Please confirm that these have been amended on your supplier account details.  Kind regards  Matt   Matthew Tibble | Senior Account Manager | E: matt@tyronex.co.uk DDI

## This was for manual labelling

In [21]:
new_df = new_df.assign(true_positives=pd.Series(1, index=new_df.index))
new_df = new_df.assign(false_positives=pd.Series(0, index=new_df.index))
new_df = new_df.assign(uncertain=pd.Series(0, index=new_df.index))
new_df = new_df.assign(notes=pd.Series(None, index=new_df.index,dtype=str))

# new_df['true_positives'] = 1
# new_df['false_positives'] = 0
# new_df['uncertain'] = 0
# new_df['notes'] = None

In [26]:
new_df.head(10)

,iter_count_list,email_id,is_part_of_thread,num_in_thread,thread_id,initial_domain,from_address,from_name,sent_to,sent_to_how_many,bcc,cc,reply_to_address,internal_date,category_id,category_slug,has_sort_code,sort_code,has_attachment,mentions_bank,mentions_change,mentions_account_specifics,mentions_statement,mentions_invoice,mentions_fraud,has_attention_ext_flag,has_alert_attach_flag,subject,attachments,email_text_body
36,13,186977,True,1,186977_t,evo-group.co.uk,Matt Tibble,Matt Tibble,"[ Matt.Hales@evo-group.co.uk , Robert.Mitchel...",5,None,None,{},13 September 2021 09:49,35,bank-detail-change,False,None,False,True,True,False,False,False,False,False,False,Tyronex Bank Account Change,None,Hi All Please find attached our new bank a...
417,356,282251,True,2,282251_t,precisionproco.co.uk,Simon.Hemingway@precisionproco.co.uk,Simon.Hemingway@precisionproco.co.uk,{purchase.ledger@evo-group.co.uk},1,None,None,{},2022-05-24 06:21:26+00,67,invoices-subject-line-pdf,False,None,False,True,True,False,False,True,True,True,False,FW: Invoice No. 666204,None,Attention: This email originated outside of ou...
419,357,282251,True,2,282251_t,precisionproco.co.uk,Simon.Hemingway@precisionproco.co.uk,Simon.Hemingway@precisionproco.co.uk,{purchase.ledger@evo-group.co.uk},1,None,None,{},2022-05-24 06:21:26+00,100,invoices,False,None,False,True,True,False,False,True,True,True,False,FW: Invoice No. 666204,None,Attention: This email originated outside of ou...
695,610,282399,False,0,False,premvan.com,Accounts@premvan.com,Accounts@premvan.com,"{Matt.Hales@evo-group.co.uk,Amy.Mitchell@banne...",2,{},{},{Accounts@premvan.com},2022-05-24 10:27:50+00,67,invoices-subject-line-pdf,True,"""09-02-22""",True,True,True,True,False,False,False,True,False,Premier Vanguard Invoice,Sales Invoice-VANGUARD_SINV00455416.PDF,Attention: This email originated outside of ...
696,611,282399,False,0,False,premvan.com,Accounts@premvan.com,Accounts@premvan.com,"{Matt.Hales@evo-group.co.uk,Amy.Mitchell@banne...",2,{},{},{Accounts@premvan.com},2022-05-24 10:27:50+00,100,invoices,True,"""09-02-22""",True,True,True,True,False,False,False,True,False,Premier Vanguard Invoice,Sales Invoice-VANGUARD_SINV00455416.PDF,Attention: This email originated outside of ...
707,622,282405,False,0,False,premvan.com,Accounts@premvan.com,Accounts@premvan.com,"{Matt.Hales@evo-group.co.uk,Amy.Mitchell@banne...",2,{},{},{Accounts@premvan.com},2022-05-24 10:27:51+00,67,invoices-subject-line-pdf,True,"""09-02-22""",True,True,True,True,False,False,False,True,False,Premier Vanguard Invoice,Sales Invoice-VANGUARD_SINV00455417.PDF,Attention: This email originated outside of ...
708,623,282405,False,0,False,premvan.com,Accounts@premvan.com,Accounts@premvan.com,"{Matt.Hales@evo-group.co.uk,Amy.Mitchell@banne...",2,{},{},{Accounts@premvan.com},2022-05-24 10:27:51+00,100,invoices,True,"""09-02-22""",True,True,True,True,False,False,False,True,False,Premier Vanguard Invoice,Sales Invoice-VANGUARD_SINV00455417.PDF,Attention: This email originated outside of ...
751,663,282428,False,0,False,pm-labels.co.uk,accountsmacc@pm-labels.co.uk,accountsmacc@pm-labels.co.uk,{Purchase.Ledger@evo-group.co.uk},1,{},{},{},2022-05-24 11:16:23+00,67,invoices-subject-line-pdf,False,None,False,True,True,False,False,True,False,True,False,Invoice,None,Attention: This email originated outside of ou...
752,664,282428,False,0,False,pm-labels.co.uk,accountsmacc@pm-labels.co.uk,accountsmacc@pm-labels.co.uk,{Purchase.Ledger@evo-group.co.uk},1,{},{},{},2022-05-24 11:16:23+00,100,invoices,False,None,False,True,True,False,False,True,False,True,False,Invoice,None,Attention: This email originated outside of ou...
780,678,282436,False,0,False,sewellwallis.co.uk,Kirsty.Teather@sewellwallis.co.uk,Kirsty.Teather@sewellwallis.co.uk,{purchase.ledger@evo-group.co.uk},1,{},{},{},2022-05-24 11:58:03+00,35,bank-detail-change,False,None,True,True,True,False,False,False,False,True,False,Change of bank details,Change of Company Bank Details.pdf,Attention: This

# Export data

In [172]:
this_file_csv = 'data/Step2_Data/labelled_emails_filtered.csv'

In [173]:
new_df.to_csv(this_file_csv)

## Read more examples

In [ ]:
i = 1
for row in new_df[0:100].itertuples():
    print("#" * 70, "\n\n")
    print(f'email index: {i}')
    i += 1
    print(f'subject line:\t{row.subject}')
    print(f'received from:\t{row.from_address}')
    print(f'is part of thread:\t{bool(row.is_part_of_thread)}')
    #print(f'num in thread:\t\t{int(row.num_in_thread)}')
    print(f'has attachment:\t\t{row.has_attachment}')
    print(f'has sort code:\t\t{row.has_sort_code}')
    print(f'mentions bank:\t\t{row.mentions_bank}')
    print(f'mentions b details:\t{row.mentions_account_specifics}')
    print(f'mentions change:\t{row.mentions_change}')
    #print(f'sent to:\t{row.sent_to}')
    print(f'sent to {row.sent_to_how_many} people')
    print(f'date sent:\t{row.internal_date}')
    print(f'slug:\t\t{row.category_slug}\n')
    print(f'{row.email_text_body}\n')

In [ ]:
# new_df.to_csv('data/Step2_Data/initial_bank_change_emails.csv')

In [ ]:
total_emails = 55199+56979+57668+131759+83226+38894
total_emails

In [ ]:
initial_filter = 722+651+626+356+567+140
initial_filter

In [ ]:
perc = (initial_filter*100)/total_emails
perc

In [ ]:
total_emails/100 # 1 percent of all emails is:

# merge dataframes here

In [181]:
import pandas as pd

In [183]:
df1 = pd.read_csv("data/Step2_Data/pot_new_targets_run1.csv")
df2 = pd.read_csv("data/Step2_Data/pot_new_targets_run2.csv")
df3 = pd.read_csv("data/Step2_Data/pot_new_targets_run3.csv")
df4 = pd.read_csv("data/Step2_Data/pot_new_targets_run4.csv")
df5 = pd.read_csv("data/Step2_Data/pot_new_targets_run5.csv")
df6 = pd.read_csv("data/Step2_Data/pot_new_targets_run6.csv")

In [184]:
tot_df = pd.concat([df1,df2,df3,df4,df5,df6],ignore_index=True)



In [185]:
tot_df.to_csv('data/Step2_Data/merged_potential_targets_v2.csv')

## Save as pickle?

In [ ]:
df = pd.read_pickle("bank_change_emails_split_newfeatures.pkl")

In [ ]:
i = 1
for row in df[0:100].itertuples():
    print("#" * 70, "\n\n")
    print(f'email index: {i}')
    i += 1
    print(f'subject line:\t{row.subject}')
    print(f'received from:\t{row.from_address}')
    print(f'is part of thread:\t{bool(row.is_part_of_thread)}')
    #print(f'num in thread:\t\t{int(row.num_in_thread)}')
    print(f'has attachment:\t\t{row.has_attachment}')
    print(f'has sort code:\t\t{row.has_sort_code}')
    print(f'mentions bank:\t\t{row.mentions_bank}')
    print(f'mentions b details:\t{row.mentions_account_specifics}')
    print(f'mentions change:\t{row.mentions_change}')
    #print(f'sent to:\t{row.sent_to}')
    print(f'sent to {row.sent_to_how_many} people')
    print(f'date sent:\t{row.internal_date}')
    print(f'slug:\t\t{row.category_slug}\n')
    print(f'{row.email_text_body}\n')